In [2]:
# Load libraries
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘jsonlite’


The following object is masked from ‘package:purrr’:

    flatten




In [1]:
# This cell downloads the original data from data.bs.ch and saves it in the raw folder
# It has been downloaded on 2024-03-01

# download_link <- "https://data.bs.ch/api/explore/v2.1/catalog/datasets/100138/exports/csv?lang=de&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
# download.file(download_link, destfile = "/raw/100138.csv")

In [3]:
# Load original data
original_data <- read_delim("/raw/100138.csv", delim = ";")

Rows: 1664159 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr  (14): Wanderungstyp, Wochentag, Staatsangehörigkeit, Geschlecht, Von Ko...
dbl   (7): Jahr, Monat, Kalenderwoche, Tag-Nr., Alter, Aufenthaltsdauer in J...
date  (2): Datum, Startdatum Woche

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
# Peek into the original data
head(original_data)

Wanderungstyp,Datum,Jahr,Monat,Kalenderwoche,Startdatum Woche,Tag-Nr.,Wochentag,Staatsangehörigkeit,Geschlecht,⋯,Von Land,Von Kanton,Von Gemeinde,Von Wohnviertel,Nach Kontinent,Nach Land,Nach Kanton,Nach Gemeinde,Nach Wohnviertel,Anzahl
<chr>,<date>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Zuzug,2003-08-08,2003,8,32,2003-08-04,220,Fr,Ausländer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Riehen,Riehen,1
Zuzug,2003-08-08,2003,8,32,2003-08-04,220,Fr,Ausländer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Riehen,Riehen,1
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Ausländer,M,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,Breite,1
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Ausländer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,Gundeldingen,1
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Schweizer,M,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,St. Alban,2
Zuzug,2003-08-07,2003,8,32,2003-08-04,219,Do,Schweizer,W,⋯,NA,NA,NA,NA,Schweiz,Schweiz,BS,Basel,Iselin,1


In [5]:
# Save new dataframe with less columns
data <- select(original_data,
  -"Datum",
  -"Kalenderwoche",
  -"Startdatum Woche",
  -"Tag-Nr.",
  -"Wochentag",
  -"Aufenthaltsdauer in Jahren",
  -"Von Gemeinde",
  -"Nach Gemeinde"
)

In [6]:
# Generate factors
columns <- c(
  "Wanderungstyp",
  "Staatsangehörigkeit",
  "Geschlecht",
  "Von Kontinent",
  "Von Land",
  "Von Kanton",
  "Von Wohnviertel",
  "Nach Kontinent",
  "Nach Land",
  "Nach Kanton",
  "Nach Wohnviertel"
)
data[, columns] <- lapply(data[, columns], as.factor)

In [7]:
# Remove spaces from column names
colnames(data) <- make.names(colnames(data))

In [8]:
# Rename column names
data <- rename(data,
  Staatsangehoerigkeit = Staatsangehörigkeit,
  VonKontinent = Von.Kontinent,
  VonLand = Von.Land,
  VonKanton = Von.Kanton,
  VonWohnviertel = Von.Wohnviertel,
  NachKontinent = Nach.Kontinent,
  NachLand = Nach.Land,
  NachKanton = Nach.Kanton,
  NachWohnviertel = Nach.Wohnviertel
)

In [9]:
# Sort data
data <- arrange(data, Jahr, Monat)

In [10]:
# Repeat each row n times if Anzahl is > 1
# For the final visualization, we want to have an individual data point for each migration

# Create a vector of row indices to repeat
row_indices <- rep(seq_len(nrow(data)), ifelse(data$Anzahl > 1, data$Anzahl - 1, 0))

# Create the expanded dataframe by indexing the original dataframe
expanded_data <- data[row_indices, ]

In [11]:
# Bind the copied rows to the project data
data <- rbind(data, expanded_data)

In [12]:
# Re-sort the project data
data <- arrange(data, Jahr, Monat, Alter)

In [13]:
# Remove the Anzahl column, it's no longer needed
data <- select(data, -"Anzahl")

In [14]:
# Peek into the project data
head(data)
tail(data)
summary(data)

Wanderungstyp,Jahr,Monat,Staatsangehoerigkeit,Geschlecht,Alter,VonKontinent,VonLand,VonKanton,VonWohnviertel,NachKontinent,NachLand,NachKanton,NachWohnviertel
<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
Umzug,1985,1,Ausländer,W,0,Schweiz,Schweiz,BS,Klybeck,Schweiz,Schweiz,BS,Klybeck
Wegzug,1985,1,Ausländer,M,0,Schweiz,Schweiz,BS,St. Alban,unbekannt,NA,NA,NA
Umzug,1985,1,Schweizer,W,1,Schweiz,Schweiz,BS,Klybeck,Schweiz,Schweiz,BS,Gotthelf
Umzug,1985,1,Ausländer,M,1,Schweiz,Schweiz,BS,Iselin,Schweiz,Schweiz,BS,St. Alban
Zuzug,1985,1,Schweizer,M,1,Schweiz,NA,NA,NA,Schweiz,Schweiz,BS,Riehen
Zuzug,1985,1,Schweizer,W,1,Schweiz,NA,NA,NA,Schweiz,Schweiz,BS,St. Johann


Wanderungstyp,Jahr,Monat,Staatsangehoerigkeit,Geschlecht,Alter,VonKontinent,VonLand,VonKanton,VonWohnviertel,NachKontinent,NachLand,NachKanton,NachWohnviertel
<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
Umzug,2023,9,Schweizer,M,94,Schweiz,Schweiz,BS,Am Ring,Schweiz,Schweiz,BS,Gundeldingen
Umzug,2023,9,Schweizer,W,96,Schweiz,Schweiz,BS,Bachletten,Schweiz,Schweiz,BS,Am Ring
Wegzug,2023,9,Schweizer,W,97,Schweiz,Schweiz,BS,Gundeldingen,Schweiz,Schweiz,VS,NA
Umzug,2023,9,Schweizer,W,98,Schweiz,Schweiz,BS,Iselin,Schweiz,Schweiz,BS,Am Ring
Umzug,2023,9,Schweizer,W,102,Schweiz,Schweiz,BS,Riehen,Schweiz,Schweiz,BS,Am Ring
Umzug,2023,9,Ausländer,M,123,Schweiz,Schweiz,BS,Matthäus,Schweiz,Schweiz,BS,St. Johann


 Wanderungstyp        Jahr          Monat        Staatsangehoerigkeit
 Umzug :751153   Min.   :1985   Min.   : 1.000   Ausländer:822157    
 Wegzug:465174   1st Qu.:1994   1st Qu.: 4.000   Schweizer:879146    
 Zuzug :484976   Median :2005   Median : 7.000                       
                 Mean   :2004   Mean   : 6.607                       
                 3rd Qu.:2014   3rd Qu.: 9.000                       
                 Max.   :2023   Max.   :12.000                       
                                                                     
 Geschlecht     Alter                           VonKontinent    
 M:890396   Min.   :  0.00   Schweiz                  :1446070  
 W:810907   1st Qu.: 23.00   Europa (ohne Schweiz)    : 190562  
            Median : 30.00   Asien                    :  17914  
            Mean   : 32.28   Unbekannt                :  17271  
            3rd Qu.: 40.00   Nordamerika              :  16737  
            Max.   :138.00   Lateinamerika und Kar

In [16]:
# Save project data as CSV
write_csv(data, "tidy/migration.csv")

In [16]:
# In order to host the data on GitHub, we need to divide it into chunks as GitHub has a file size limit of approx. 50MB

# Read final data
final_data <- read_delim("tidy/migration.csv", delim = ",")

Rows: 1701303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): Wanderungstyp, Staatsangehoerigkeit, Geschlecht, VonKontinent, Von...
dbl  (3): Jahr, Monat, Alter

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [18]:
# Create chunks of data
chunk_size <- 300000
chunks <- split(final_data, (seq_len(nrow(final_data)) - 1) %/% chunk_size)

In [21]:
# Save each chunk as a separate csv file
output_dir <- "tidy/chunks/"
for (i in seq_along(chunks)) {
  write_csv(chunks[[i]], paste0(output_dir, "migration_", i, ".csv"))
}